# Data retrieval from localbitcoins API

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone

In [2]:
def make_request(url_base,url):
    r = requests.get(url_base+url)
    file_json = r.json()
    return file_json

def json_to_pandas(file_json:dict):
    return pd.DataFrame.from_dict(file_json)


### Calculation exchange rate of reference to further math on pct_change:
def exchange_reference(df, date_reference):
    
    memory = {}
        
    for currency_code in df["currency_code"].unique():
        
        if currency_code != "VED":
            date_reference = date_reference
        else:
            date_reference = "2021-10-10"

    
        ## Select currency for calculations:
        df_exchange = df[df['currency_code'] == currency_code ]

        ## Select date to calculate reference

        df_date_reference = df_exchange.loc[(df_exchange.date).dt.date == pd.Timestamp(date_reference)]

        ### Averare exchange rate for day of reference:

        implicit_exchange_reference = df_date_reference.implicit_exchange.mean()
        
        memory[currency_code] = implicit_exchange_reference

    ### Percentual Variation

#     equation = ((df_exchange.implicit_exchange / implicit_exchange_reference)-1)*100
    
    
    return memory

## Currencies

In [3]:
url_base = 'https://localbitcoins.com/'
# Currencies:
url = "api/currencies"
currency_code = make_request(url_base,url)
df_currency_code = json_to_pandas(currency_code['data'])
# df_currency_code.info()
# df_currency_code.head()

In [4]:
## Extract country information related to currency
currency_name = []
currency_is_altcoin = []
for index, row in df_currency_code.iterrows():
    currency_name.append(row[0]["name"])
    currency_is_altcoin.append(row[0]["altcoin"])
    
## Reorganize on df

df_currency_code['currencies'] = currency_name
df_currency_code['altcoin'] = currency_is_altcoin
df_currency_code = df_currency_code.reset_index()

df_currency_code.head()

,index,currencies,currency_count,altcoin
0,ADA,Cardano (ADA),185,True
1,AED,United Arab Emirates Dirham (AED),185,False
2,AFN,Afghan Afghani (AFN),185,False
3,ALL,Albanian Lek (ALL),185,False
4,AMD,Armenian Dram (AMD),185,False


### Average price of btc by country (in local currency)

In [5]:
url = "bitcoinaverage/ticker-all-currencies/"
price_btc = make_request(url_base,url)
time_stamp = datetime.now().timestamp()

In [6]:
df_price_btc = json_to_pandas(price_btc)

#transpose data frame. Index is now by Country:

df_price_btc = df_price_btc.T
df_price_btc['time_stamp'] = time_stamp
df_price_btc = df_price_btc.reset_index()
print(df_price_btc.info())
df_price_btc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       84 non-null     object 
 1   volume_btc  84 non-null     object 
 2   rates       84 non-null     object 
 3   avg_1h      27 non-null     object 
 4   avg_6h      50 non-null     object 
 5   avg_12h     59 non-null     object 
 6   avg_24h     84 non-null     object 
 7   time_stamp  84 non-null     float64
dtypes: float64(1), object(7)
memory usage: 5.4+ KB
None


,index,volume_btc,rates,avg_1h,avg_6h,avg_12h,avg_24h,time_stamp
0,ZAR,2.08822577,{'last': '749299.79'},754912.43,752357.54,752774.21,752845.83,1.640938e+09
1,NGN,2.10597978,{'last': '26112531.27'},26443734.57,26465277.41,26619941.12,26533869.03,1.640938e+09
2,IDR,0.02000033,{'last': '645305616.74'},NaN,644427889.81,644427889.81,699988450.19,1.640938e+09
3,CNY,2.43137189,{'last': '302289.83'},303982.48,296042.86,296670.68,302081.65,1.640938e+09
4,RUB,5.95559893,{'last': '3433007.65'},3490555.91,3528943.97,3521273.19,3529607.65,1.640938e+09


## Merge and clean dataframe

In [7]:
df_market = df_price_btc.merge(df_currency_code, how='right')

# Drop the rows with NAN values in time_stamp
df_market.dropna(subset=['time_stamp'], inplace=True)

# Add date from time_stamp
df_market['date'] = datetime.fromtimestamp(time_stamp, tz=timezone.utc)
df_market = df_market[['time_stamp','date','index','currencies','volume_btc','avg_24h','avg_12h','avg_6h','avg_1h','altcoin']]
df_market.rename(columns = {'index':'currency_code'}, inplace = True)
print(df_market.info())
df_market.head(10)


### I should include some data formatting so info is stored in the right format

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 183
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   time_stamp     84 non-null     float64            
 1   date           84 non-null     datetime64[ns, UTC]
 2   currency_code  84 non-null     object             
 3   currencies     84 non-null     object             
 4   volume_btc     84 non-null     object             
 5   avg_24h        84 non-null     object             
 6   avg_12h        59 non-null     object             
 7   avg_6h         50 non-null     object             
 8   avg_1h         27 non-null     object             
 9   altcoin        84 non-null     bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), object(7)
memory usage: 6.6+ KB
None


,time_stamp,date,currency_code,currencies,volume_btc,avg_24h,avg_12h,avg_6h,avg_1h,altcoin
0,1.640938e+09,2021-12-31 08:14:54.337475+00:00,ADA,Cardano (ADA),0.00031772,35502.96,NaN,NaN,NaN,True
1,1.640938e+09,2021-12-31 08:14:54.337475+00:00,AED,United Arab Emirates Dirham (AED),0.30029001,173578.20,NaN,NaN,NaN,False
6,1.640938e+09,2021-12-31 08:14:54.337475+00:00,AOA,Angolan Kwanza (AOA),0.00638157,34897368.51,NaN,NaN,NaN,False
7,1.640938e+09,2021-12-31 08:14:54.337475+00:00,ARS,Argentine Peso (ARS),0.39637491,9963516.49,10219219.01,10273378.51,NaN,False
8,1.640938e+09,2021-12-31 08:14:54.337475+00:00,AUD,Australian Dollar (AUD),0.19555665,67451.76,65914.47,63156.83,62961.66,False
15,1.640938e+09,2021-12-31 08:14:54.337475+00:00,BDT,Bangladeshi Taka (BDT),0.00710356,4328815.41,3926296.23,3926296.23,NaN,False
16,1.640938e+09,2021-12-31 08:14:54.337475+00:00,BGN,Bulgarian Lev (BGN),0.00215481,92815.61,NaN,NaN,NaN,False
21,1.640938e+09,2021-12-31 08:14:54.337475+00:00,BOB,Bolivian Boliviano (BOB),0.06667690,342365.50,341878.03,343642.61,NaN,False
22,1.640938e+09,2021-12-31 08:14:54.337475+00:00,BRL,Brazilian Real (BRL),0.48911885,273981.63,268479.18,273164.07,NaN,False
25,1.640938e+09,2021-12-31 08:14:54.337475+00:00,BWP,Botswanan Pula (BWP),0.00076237,655849.52,NaN,NaN,NaN,False


In [8]:
## Adding avg_24h exchange rate USD/BTC for inmediate comparison:

df_market["avg_24h_usd"] = df_market[df_market['currency_code']=="USD"].avg_24h.tolist()[0]

In [9]:
### Formatting data types:

# Float values:
for column_label in ['volume_btc','avg_24h','avg_12h','avg_6h','avg_1h','avg_24h_usd']:

    df_market[column_label] = pd.to_numeric(df_market[column_label])

df_market.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 183
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   time_stamp     84 non-null     float64            
 1   date           84 non-null     datetime64[ns, UTC]
 2   currency_code  84 non-null     object             
 3   currencies     84 non-null     object             
 4   volume_btc     84 non-null     float64            
 5   avg_24h        84 non-null     float64            
 6   avg_12h        59 non-null     float64            
 7   avg_6h         50 non-null     float64            
 8   avg_1h         27 non-null     float64            
 9   altcoin        84 non-null     bool               
 10  avg_24h_usd    84 non-null     float64            
dtypes: bool(1), datetime64[ns, UTC](1), float64(7), object(2)
memory usage: 7.3+ KB


In [10]:
## Calculating implicit exchange rate:
df_market['implicit_exchange'] = df_market['avg_24h'].divide(df_market["avg_24h_usd"], fill_value = None)

In [11]:
# Drop the rows with NAN values in time_stamp
df_market.dropna(subset=['time_stamp'], inplace=True)

In [12]:
df_market.head(2)

,time_stamp,date,currency_code,currencies,volume_btc,avg_24h,avg_12h,avg_6h,avg_1h,altcoin,avg_24h_usd,implicit_exchange
0,1.640938e+09,2021-12-31 08:14:54.337475+00:00,ADA,Cardano (ADA),0.000318,35502.96,NaN,NaN,NaN,True,48444.58,0.732857
1,1.640938e+09,2021-12-31 08:14:54.337475+00:00,AED,United Arab Emirates Dirham (AED),0.300290,173578.20,NaN,NaN,NaN,False,48444.58,3.583026


## Save information in .csv and sql database

In [13]:
# Saving csv by timestamp:
df_market.to_csv(f'./data/csv/{time_stamp}.csv',index=False)
# Saving historical data by appending:
df_market.to_csv(f'./data/csv/historical_data.csv',index=False, mode = 'a')

In [14]:
# Append information on sqlite database

from sqlalchemy import Column, Integer, Float, String, DateTime,Boolean, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///data/CurrenciesDataBase.db')
print(type(engine))
df_market.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

<class 'sqlalchemy.engine.base.Engine'>


#### Calculation of pct_change taking 2021-10-03 as date of reference

In [15]:
# path_to_currencies = "sqlite:///data/CurrenciesDataBase.db"
# df_market = pd.read_sql('currencies_vs_btc', path_to_currencies)

# ### The pct calculations are not properly saved on the dataset unless I upload one previously saved. Why?
# # The pipeline defined solve the issue by saving the calculated data on a new dataset but the calculations
# # are done in the full dataset everytime the code runs.
# # This practice is not scalable

# # The pct could be calculated on the web interface for a given date selected by the users but this implies 
# # more processing time

In [16]:
# df = df_market.groupby(by=["date","currency_code"]).mean()
# df.reset_index(inplace=True)

# ### Dataset with implicit exchange of reference by currency:
# date_reference = "2021-10-03"
# exchange = exchange_reference(df, date_reference)
# exchange = pd.DataFrame.from_dict(exchange, orient="index")

# df['pct'] = df.apply(lambda x : ((x['implicit_exchange']/exchange.loc[x["currency_code"]])-1)*100, axis = 1 )

# ### Drop Null values:

# df.dropna(subset=['pct'], inplace=True)

In [17]:
# df.info()

In [18]:
# ### Saving on DB Version 2

# engine = create_engine('sqlite:///data/CurrenciesDataBase_V2.db')
# print(type(engine))
# df.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

In [19]:
now = dt.datetime.now()
print("Last Execution: ")
print(str(now))
print('Completed')

Last Execution: 
2021-12-31 09:14:54.843525
Completed
